We will use a new package ``mlxtend`` for association rules mining. Check the [documentation](https://rasbt.github.io/mlxtend/) for details.  

You may need to install the package first with the following code. 

```python
pip install mlxtend
```

# 1. Data Preprocessing

We sampled 9958 Tweets with two or more food/drink emojis, each tweets can be considered as a itemset for emojis.   

- Let's start by loading the dataset and inspect its first few lines.

In [ ]:
import pandas as pd

tweets_df = pd.read_csv("food_drink_emoji_tweets.txt", sep="\t", header=None)  # tab separated data, ignore header

tweets_df.columns = ['text']     # rename the column name  

tweets_df.head()

### 1.1 Extract food/drink emojis in each tweet

Each row is a tweet/post. Let's extract the emojis that appear in each post as an itemset. 

- We are only interested in emojis that are food and drinks.

**Step 1: create a emoji set**

In [ ]:
emoji_list = "🍇🍈🍉🍊🍋🍌🍍🥭🍎🍏🍐🍑🍒🍓🥝🍅🥥🥑🍆🥔🥕🌽🌶🥒🥬🥦🍄🥜🌰🍞🥐🥖🥨🥯🥞🧀🍖🍗🥩🥓🍔🍟🍕🌭🥪🌮🌯🥙🥚🍳🥘🍲🥣🥗🍿🧂🥫🍱🍘🍙🍚🍛🍜🍝🍠🍢🍣🍤🍥🥮🍡🥟🥠🥡🦀🦞🦐🦑🍦🍧🍨🍩🍪🎂🍰🧁🥧🍫🍬🍭🍮🍯🍼🥛☕🍵🍶🍾🍷🍸🍹🍺🍻🥂🥃"
        
emoji_set = set(emoji_list)  # convert the sequence of emjois as itemset

#emoji_set    # uncomment to see the result

**Step 2: extract emoji from each tweet as a set**

In [ ]:
def extract_unique_emoji(text):
    emoji_list_per_tweet = [emoji for emoji in text if emoji in emoji_set]       # list comphrehension
    emoji_set_per_tweet = set(emoji_list_per_tweet)                              # convert the list as a set 
    return  emoji_set_per_tweet                                        

tweets_df['emojis'] = tweets_df['text'].apply(extract_unique_emoji)        # apply the function to column 'text'
tweets_df.head()

### 1.2 Transform emoji items into matrix 

Before we use the `mlxtend` for frequent itemset mining, we need to encode the items into a matrix where (1) each row represents an itemset (tweet) and (2) each column represents an item, and (3) each cell record whether an item is in the itemset (tweet) or not.  

- Let's use the `TransactionEncoder` function from `mlxtend.preprocessing`. Check [documentation](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.preprocessing/#transactionencoder) for details.
- Alternatively, you may also use the `MultiLabelBinarizer` from `scikit-learn`. Check [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html) for details.

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()

arr = encoder.fit_transform(tweets_df['emojis'])   # return a matrix

arr   # a 2D array of boolean values 

In [ ]:
# convert as dataframe

emoji_df = pd.DataFrame(data = arr,  columns = encoder.columns_)

emoji_df

# 2.  Frequent Itemsets Mining

## 2.1 Find frequent itemsets 

Now, let's apply the Apriori Algorithm to the emoji dataset.

Call the `apriori` API from `mlxtend.frequent_patterns` and specify the minimal support we want. Check the [User Guide](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) and [Documentation](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/#apriori) for more details.

- set `min_support=0.005`, i.e., half percent of the tweets. 
- set ``use_colnames=True`` to convert column indices of each item (default) into the respective item names for readability.

In [ ]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df = emoji_df, min_support = 0.005, use_colnames = True)

frequent_itemsets

## 2.2 Sort frequent itemsets by their support values

In [ ]:
frequent_itemsets.sort_values(by = 'support',ascending = False)

<font color=red>***Exercise 1: Your Codes Here***</font>  

**Step 1**: How many itemsets appears in at least 10% of the posts?

**Step 2**: Extract frequent itemsets with at least two 2 items.

- Hint: use `pandas.Series.apply` function together with the `len` to check the number of items in each itemset first.

# 3.  Association Rules 

In this section, we will practice how to evaluate the relationship between frequent itemsets. 

- Let's call the `association_rules` API from `mlxtend.frequent_patterns`. Visit the [User Guide](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)  and [Documentation](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/#association_rules) for more details.

The association rules represent the two relationship between to frequent itemsets: an `antecedent` item(set) and a `consequent` item(set). 

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(df = frequent_itemsets,     # frequent itemsets generated by apriori algorihm
                          metric = "lift",         
                          min_threshold = 1)          # min lift = 1

rules.sort_values(by = 'lift',ascending = False)

In the returned data frame, each row examines a pair of `antecedent` (X) -> `consequent` (Y). 

- While the `Antecedent support` and `consequent support` measure *P*(X) and *P*(Y) respectively, the `support` measures the join probability *P*(X, Y).  

Other metrics such as `confidence`, `lift`, `leverage`, and `conviction` can be derived from the three support values. 

For example, $$\text{confidence}=\frac{\text{support}}{\text{antecedent\_support}},$$ and 


$$\text{lift} =\frac{\text{confidence}}{\text{consequent\_support}}=\frac{\text{support}}{\text{antecedent\_support} \times \text{consequent\_support}},$$ 

and 

$$\text{leverage}= \text{support} - \text{antecedent\_support} * \text{consequent\_support}, $$ 
 

# 4.  Itemset Similarity

Pattern and similarity are two basic outputs of data mining. So far, we have been playing with patterns - frequent itemsets and association rules can all be seen as "patterns". In the last part, let's work on itemset similarities.
 
**Jaccard similarity** is a simple but powerful measurement of itemset similarity, defined as follows:
$$\text{Jaccard\_similarity(A, B)} = \frac{|A\cap B|}{|A\cup B|}$$



In [ ]:
def jaccard_similarity(A, B):
    intersection = A.intersection(B)
    union = A.union(B)
    jaccard_sim = len(intersection)/len(union)
    return jaccard_sim

Calculate the jaccard similarity of two sets.

In [ ]:
fruit_A = set(['🍇', '🍒', '🍎'])
fruit_B = set(['🍊', '🍒', '🍎'])

jaccard_similarity(fruit_A, fruit_B)

**Which tweet is most similiar to the first tweet post in terms of the set of food and drink emojis used?** 


**Step 1**: define a function which returns the jaccard similarity between the first itemset and any itemsets.

In [ ]:
def jaccard_one(X):
    first_tweet = tweets_df.loc[0, 'emojis']
    jaccard_sim = jaccard_similarity(first_tweet, X)
    return jaccard_sim

<font color=red>***Exercise 2: Your Codes Here***</font>  

**Step 2**: calculate the jaccard similarity of the first emoji itemset and the itemsets in each row, and save the jaccard similarities a separate column named `jaccard`. 

**Step 3**: sort the dataframe by the column `jaccard` and check the top 10 rows.